# 05. Мультитаймфреймовый анализ

## Цель
Реализовать мультитаймфреймовый анализ для оценки согласованности сигналов между разными таймфреймами.

## Задачи
1. Загрузить данные для всех таймфреймов (15m, 30m, 1h, 4h, 1d)
2. Реализовать анализ согласованности сигналов
3. Реализовать Lead/Lag анализ
4. Реализовать иерархический анализ таймфреймов
5. Создать визуализацию результатов


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sys
import os

# Добавляем путь к нашим модулям
sys.path.append('../evaluation')
sys.path.append('../classifiers')

from economic_metrics import EconomicMetrics
from classifiers import MZAClassifier

print("Библиотеки загружены успешно!")
print(f"Текущая директория: {os.getcwd()}")


In [ ]:
# Загружаем данные для всех таймфреймов
print("=== ЗАГРУЗКА ДАННЫХ ДЛЯ ВСЕХ ТАЙМФРЕЙМОВ ===")

timeframes = ['15m', '30m', '1h', '4h', '1d']
data = {}

for tf in timeframes:
    try:
        file_path = f'../../indicators/data_frames/df_btc_{tf}.csv'
        df = pd.read_csv(file_path)
        df['timestamps'] = pd.to_datetime(df['timestamps'])
        df.set_index('timestamps', inplace=True)
        data[tf] = df
        print(f"✅ {tf}: {len(df)} записей, период: {df.index[0]} - {df.index[-1]}")
    except Exception as e:
        print(f"❌ Ошибка загрузки {tf}: {e}")

print(f"\nЗагружено {len(data)} таймфреймов")
print(f"Доступные таймфреймы: {list(data.keys())}")


In [ ]:
# Получаем предсказания для всех таймфреймов
print("=== ПОЛУЧЕНИЕ ПРЕДСКАЗАНИЙ ДЛЯ ВСЕХ ТАЙМФРЕЙМОВ ===")

# Создаем MZA классификатор
mza = MZAClassifier()

# Получаем предсказания для каждого таймфрейма
predictions = {}
for tf, df in data.items():
    try:
        # Обучаем на первых 70% данных
        train_size = int(len(df) * 0.7)
        train_data = df.iloc[:train_size]
        test_data = df.iloc[train_size:]
        
        # Обучаем и предсказываем
        mza.fit(train_data)
        pred = mza.predict(test_data)
        predictions[tf] = pred
        
        print(f"✅ {tf}: {len(pred)} предсказаний")
        
        # Статистика предсказаний
        unique, counts = np.unique(pred, return_counts=True)
        for regime, count in zip(unique, counts):
            regime_name = {-1: 'Медвежий', 0: 'Боковой', 1: 'Бычий'}.get(regime, f'Неизвестный_{regime}')
            percentage = count / len(pred) * 100
            print(f"    {regime_name}: {count} ({percentage:.1f}%)")
        
    except Exception as e:
        print(f"❌ Ошибка для {tf}: {e}")

print(f"\nПолучено предсказаний для {len(predictions)} таймфреймов")


In [ ]:
# Анализ согласованности сигналов между таймфреймами
print("=== АНАЛИЗ СОГЛАСОВАННОСТИ СИГНАЛОВ ===")

def calculate_signal_consistency(predictions_dict):
    """Вычисление согласованности сигналов между таймфреймами"""
    timeframes = list(predictions_dict.keys())
    consistency_matrix = pd.DataFrame(index=timeframes, columns=timeframes)
    
    for tf1 in timeframes:
        for tf2 in timeframes:
            if tf1 != tf2:
                # Находим общие периоды (упрощенная версия)
                min_length = min(len(predictions_dict[tf1]), len(predictions_dict[tf2]))
                pred1 = predictions_dict[tf1][:min_length]
                pred2 = predictions_dict[tf2][:min_length]
                
                # Вычисляем корреляцию
                correlation = np.corrcoef(pred1, pred2)[0, 1]
                consistency_matrix.loc[tf1, tf2] = correlation
            else:
                consistency_matrix.loc[tf1, tf2] = 1.0
    
    return consistency_matrix

# Вычисляем матрицу согласованности
consistency_matrix = calculate_signal_consistency(predictions)

print("📊 Матрица согласованности сигналов:")
print(consistency_matrix.round(4))

# Находим наиболее согласованные пары
print("\n🔗 Наиболее согласованные пары таймфреймов:")
for tf1 in consistency_matrix.index:
    for tf2 in consistency_matrix.columns:
        if tf1 != tf2:
            correlation = consistency_matrix.loc[tf1, tf2]
            print(f"  {tf1} ↔ {tf2}: {correlation:.4f}")


In [ ]:
# Lead/Lag анализ
print("=== LEAD/LAG АНАЛИЗ ===")

def analyze_lead_lag(predictions_dict, reference_tf='1d'):
    """Анализ опережающих и запаздывающих сигналов"""
    lead_lag_results = {}
    
    # Используем дневной таймфрейм как эталон
    if reference_tf not in predictions_dict:
        print(f"❌ Эталонный таймфрейм {reference_tf} не найден")
        return {}
    
    reference_predictions = predictions_dict[reference_tf]
    
    for tf, pred in predictions_dict.items():
        if tf != reference_tf:
            # Вычисляем корреляции с различными лагами
            lags = range(-5, 6)  # от -5 до +5 периодов
            correlations = []
            
            for lag in lags:
                if lag > 0:
                    # Текущий таймфрейм опережает эталонный
                    if len(pred) > lag and len(reference_predictions) > lag:
                        corr = np.corrcoef(pred[:-lag], reference_predictions[lag:])[0, 1]
                    else:
                        corr = 0
                elif lag < 0:
                    # Эталонный опережает текущий таймфрейм
                    if len(pred) > abs(lag) and len(reference_predictions) > abs(lag):
                        corr = np.corrcoef(pred[-lag:], reference_predictions[:lag])[0, 1]
                    else:
                        corr = 0
                else:
                    # Без лага
                    min_length = min(len(pred), len(reference_predictions))
                    if min_length > 0:
                        corr = np.corrcoef(pred[:min_length], reference_predictions[:min_length])[0, 1]
                    else:
                        corr = 0
                
                correlations.append(corr)
            
            lead_lag_results[tf] = {
                'lags': lags,
                'correlations': correlations,
                'best_lag': lags[np.argmax(correlations)],
                'max_correlation': max(correlations)
            }
    
    return lead_lag_results

# Выполняем Lead/Lag анализ
lead_lag_results = analyze_lead_lag(predictions, reference_tf='1d')

print("📈 Результаты Lead/Lag анализа:")
for tf, result in lead_lag_results.items():
    print(f"  {tf}:")
    print(f"    Лучший лаг: {result['best_lag']}")
    print(f"    Максимальная корреляция: {result['max_correlation']:.4f}")
    
    # Определяем тип связи
    if result['best_lag'] > 0:
        print(f"    {tf} опережает 1d на {result['best_lag']} периодов")
    elif result['best_lag'] < 0:
        print(f"    1d опережает {tf} на {abs(result['best_lag'])} периодов")
    else:
        print(f"    {tf} синхронен с 1d")


In [ ]:
# Визуализация результатов мультитаймфреймового анализа
print("=== ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ ===")

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# График 1: Матрица согласованности
ax1 = axes[0, 0]
sns.heatmap(consistency_matrix.astype(float), annot=True, cmap='RdYlBu_r', 
            center=0, ax=ax1, cbar_kws={'label': 'Корреляция'})
ax1.set_title('Матрица согласованности сигналов')
ax1.set_xlabel('Таймфрейм')
ax1.set_ylabel('Таймфрейм')

# График 2: Lead/Lag анализ
ax2 = axes[0, 1]
for tf, result in lead_lag_results.items():
    ax2.plot(result['lags'], result['correlations'], 'o-', label=tf, alpha=0.7)
ax2.set_title('Lead/Lag анализ (относительно 1d)')
ax2.set_xlabel('Лаг (периоды)')
ax2.set_ylabel('Корреляция')
ax2.legend()
ax2.grid(True, alpha=0.3)

# График 3: Максимальные корреляции
ax3 = axes[1, 0]
max_correlations = [result['max_correlation'] for result in lead_lag_results.values()]
timeframes = list(lead_lag_results.keys())
bars = ax3.bar(timeframes, max_correlations, alpha=0.7, color='skyblue')
ax3.set_title('Максимальные корреляции с 1d')
ax3.set_ylabel('Корреляция')
ax3.grid(True, alpha=0.3)

# Добавляем значения на столбцы
for bar, value in zip(bars, max_correlations):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.3f}', ha='center', va='bottom')

# График 4: Лучшие лаги
ax4 = axes[1, 1]
best_lags = [result['best_lag'] for result in lead_lag_results.values()]
colors = ['green' if lag > 0 else 'red' if lag < 0 else 'gray' for lag in best_lags]
bars = ax4.bar(timeframes, best_lags, alpha=0.7, color=colors)
ax4.set_title('Лучшие лаги относительно 1d')
ax4.set_ylabel('Лаг (периоды)')
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax4.grid(True, alpha=0.3)

# Добавляем значения на столбцы
for bar, value in zip(bars, best_lags):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{value}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("✅ Графики созданы")
